In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x105a35820>>
Traceback (most recent call last):
  File "/Users/arnavkhanduja/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [3]:
X = pd.read_csv('input_4200.csv').values  # file with 80 columns of 0s and 1s
y = pd.read_csv('mapped_snr_4200.csv').values       # file with 80 columns of SNR values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [4]:
xgb_model = XGBRegressor(n_estimators=200, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_val_preds = xgb_model.predict(X_val) * X_val
xgb_test_preds = xgb_model.predict(X_test) * X_test

xgb_val_score = r2_score(y_val, xgb_val_preds)
xgb_test_score = r2_score(y_test, xgb_test_preds)
xgb_mse = mean_squared_error(y_test, xgb_test_preds)

print(f'XGBoost Val R2 Score: {xgb_val_score}')
print(f'XGBoost Test R2 Score: {xgb_test_score}')
print(f'XGBoost Test MSE: {xgb_mse}')

XGBoost Val R2 Score: 0.9999993064160474
XGBoost Test R2 Score: 0.9999992776084149
XGBoost Test MSE: 8.066618623530787e-05


In [5]:
def predict_and_save(model, X_test, output_file):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        outputs = outputs * X_test  # Apply mask
    outputs = outputs.numpy()
    np.savetxt(output_file, outputs, delimiter=',')
'''custom_input = np.array([1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])'''
custom_input = np.array([1 if i%3==0 else 0 for i in range(80)])
#custom_input
X_test_custom = pd.read_csv('input_test_5700-6000.csv').values
input_file = 'input_test_5700-6000.csv'  # Path to the file with 300 records and 80 columns
output_file = 'predicted_snr_output.csv'


In [6]:
models = ['Random Forest', 'XGBoost']
val_scores = [ rf_val_score, xgb_val_score]
test_scores = [ rf_test_score, xgb_test_score]

plt.figure(figsize=(10, 5))
plt.scatter(models, val_scores, label='Validation Scores')
plt.scatter(models, test_scores, label='Test Scores')
#plt.yscale('log')
plt.xlabel('Model')
plt.ylabel('R2 Score')
plt.title('Model Performance Comparison')
plt.legend()
plt.show()

NameError: name 'rf_val_score' is not defined

In [14]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Load data
X = pd.read_csv('input_4200.csv').values  # file with 80 columns of 0s and 1s
y = pd.read_csv('mapped_snr_4200.csv').values       # file with 80 columns of SNR values

# Mask the output data
y_masked = y * X  # Applying mask: where X is 0, y will be set to 0

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_masked, test_size=0.2, random_state=42)

# Initialize and train XGBoost model
xgb_model = XGBRegressor(n_estimators=200, random_state=42)
xgb_model.fit(X_train, y_train)

xgb_test_preds = xgb_model.predict(X_test) * X_test  # Apply mask to the predictions
xgb_test_score = r2_score(y_test, xgb_test_preds)
xgb_mse = mean_squared_error(y_test, xgb_test_preds)
print(f'XGBoost Test R2 Score: {xgb_test_score}')
print(f'XGBoost Test MSE: {xgb_mse}')
# Predict using trained XGBoost model
test_features = pd.read_csv('input_test_5700-6000.csv', header=None).values
print(f"Test data shape: {test_features.shape}")  # Load test data
test_outputs = xgb_model.predict(test_features)
test_outputs = test_outputs * test_features  # Apply mask where input is 0, output should be 0
print("First few rows of test_outputs:")
#print(test_outputs[:5])
# Save predicted outputs to a file
np.savetxt('predicted_outputs.csv', test_outputs, delimiter=',', fmt='%f')

print("Predicted outputs saved to predicted_outputs.csv file.")


XGBoost Test R2 Score: 0.9999994217605389
XGBoost Test MSE: 6.44594240376549e-05
Test data shape: (300, 80)
First few rows of test_outputs:
Predicted outputs saved to predicted_outputs.csv file.


In [2]:
'''XGBoost Test R2 Score: 0.9999994165693391
XGBoost Test MSE: 6.50513412664856e-05'''

'XGBoost Test R2 Score: 0.9999994165693391\nXGBoost Test MSE: 6.50513412664856e-05'

In [ ]:
#GRID SEARCH
from sklearn.model_selection import GridSearchCV
param_grid = {
    'learning_rate': [0.2, 0.3, 0.4],
    'n_estimators': [100, 200, 300],
    'max_depth': [4,6,8],
    #'min_child_weight': [0, 1, 3, 5],
    #'gamma': [0, 0.1, 0.2],
    
    #'subsample': [0.5, 0.8, 1.0],
    #'colsample_bytree': [0.6, 0.8, 1.0],
    #'reg_alpha': [0, 0.1, 0.5],
    #'reg_lambda': [1, 1.5, 2],
    'random_state': [32,42,64]
}
xgb= XGBRegressor()
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)


In [15]:

# Initialize and train XGBoost model
xgb_model = XGBRegressor(n_estimators=300, random_state=32, max_depth=4, learning_rate=0.2)
xgb_model.fit(X_train, y_train)

xgb_test_preds = xgb_model.predict(X_test) * X_test  # Apply mask to the predictions
xgb_test_score = r2_score(y_test, xgb_test_preds)
xgb_mse = mean_squared_error(y_test, xgb_test_preds)
print(f'XGBoost Test R2 Score: {xgb_test_score}')
print(f'XGBoost Test MSE: {xgb_mse}')
# Predict using trained XGBoost model
test_features = pd.read_csv('input_test_5700-6000.csv', header=None).values
print(f"Test data shape: {test_features.shape}")  # Load test data
test_outputs = xgb_model.predict(test_features)
test_outputs = test_outputs * test_features  # Apply mask where input is 0, output should be 0
print("First few rows of test_outputs:")
#print(test_outputs[:5])
# Save predicted outputs to a file
np.savetxt('predicted_outputs.csv', test_outputs, delimiter=',', fmt='%f')

print("Predicted outputs saved to predicted_outputs.csv file.")


XGBoost Test R2 Score: 0.9999997513998489
XGBoost Test MSE: 2.7828871966050466e-05
Test data shape: (300, 80)
First few rows of test_outputs:
Predicted outputs saved to predicted_outputs.csv file.
